In [22]:
import pandas as pd
import numpy as np
import csv
from decimal import Decimal


In [23]:
from sklearn.model_selection import train_test_split

df = pd.read_csv("trg.csv")
df = df.rename({'class':'target'},axis=1)
training, testing = train_test_split(df, test_size=0.3, random_state=42)



In [24]:
vocab = {word for para in training['abstract'] for word in para.split()}


prior_probability = training['target'].value_counts(normalize=True).to_dict()
for key, value in prior_probability.items():
    prior_probability[key] = Decimal(str(value))
prior_probability 

{'E': Decimal('0.5457142857142857'),
 'B': Decimal('0.38785714285714284'),
 'A': Decimal('0.03535714285714286'),
 'V': Decimal('0.031071428571428573')}

In [25]:
word_orrurance = dict()
for row in training.itertuples(index = 0): 
    line = row.abstract
    target = row.target
    # print(target)
    for word in line.split():
        
        if word in word_orrurance.keys():
            if target in word_orrurance[word].keys():
                word_orrurance[word][target] += 1
            else:
                word_orrurance[word][target] = 1
        else:
            word_orrurance[word] = {"B": Decimal(0), "E":Decimal(0), "A":Decimal(0), "V":Decimal(0)}
            # word_orrurance[word] = {target:1}
            # word_orrurance[word] = {"B": 0, "E":0 , "A":0, "V":0}
            word_orrurance[word][target] += 1



data = []
for word, targets in word_orrurance.items():
    
    row = {'word': word}
    
    for i, (target, count) in enumerate(targets.items()):
        row [target] = count
    # append the row to the data list
    data.append(row)
word_orrurance_table = pd.DataFrame(data)
word_orrurance_table.fillna(0, inplace=True)

# print(word_orrurance_table.loc[word_orrurance_table['word'] == 'virus'])
# word_orrurance_table.head()

word_orrurance


{'the': {'B': Decimal('12290'),
  'E': Decimal('17525'),
  'A': Decimal('1342'),
  'V': Decimal('1214')},
 'structure': {'B': Decimal('120'),
  'E': Decimal('289'),
  'A': Decimal('28'),
  'V': Decimal('19')},
 'and': {'B': Decimal('6744'),
  'E': Decimal('8996'),
  'A': Decimal('766'),
  'V': Decimal('512')},
 'expression': {'B': Decimal('176'),
  'E': Decimal('743'),
  'A': Decimal('7'),
  'V': Decimal('9')},
 'of': {'B': Decimal('10883'),
  'E': Decimal('12756'),
  'A': Decimal('987'),
  'V': Decimal('886')},
 'alpha-amylase-encoding': {'B': Decimal('0'),
  'E': Decimal('3'),
  'A': Decimal('0'),
  'V': Decimal('0')},
 'gene': {'B': Decimal('1317'),
  'E': Decimal('1921'),
  'A': Decimal('173'),
  'V': Decimal('98')},
 'ramy2a': {'B': Decimal('0'),
  'E': Decimal('2'),
  'A': Decimal('0'),
  'V': Decimal('0')},
 'are': {'B': Decimal('1385'),
  'E': Decimal('1709'),
  'A': Decimal('326'),
  'V': Decimal('95')},
 'described': {'B': Decimal('39'),
  'E': Decimal('128'),
  'A': Decimal(

In [26]:
total_B = word_orrurance_table['B'].sum()
total_E = word_orrurance_table['E'].sum()
total_A = word_orrurance_table['A'].sum()
total_V = word_orrurance_table['V'].sum()

conditional_probability = word_orrurance_table.copy()
conditional_probability['B'] = (conditional_probability['B'] / (total_B)).apply(Decimal)
conditional_probability['E'] = (conditional_probability['E'] / (total_E)).apply(Decimal)
conditional_probability['A'] = (conditional_probability['A'] / (total_A)).apply(Decimal)
conditional_probability['V'] = (conditional_probability['V'] / (total_V)).apply(Decimal)


conditional_probability_dict = word_orrurance

# for keys in conditional_probability_dict

In [27]:
def naive_Bayes(vocab,conditional_probability, prior_probability, conditional_probability_dict,testing):
    predictions = []
    row_nm = 0
    for row in testing.itertuples(index = 0):
        row_nm += 1
        print(row_nm)
        line = row.abstract
        prediction = predict(vocab, conditional_probability,prior_probability,conditional_probability_dict,line)
        print(prediction, row.target)
        predictions.append(prediction)
    testing['prediction'] = predictions

def predict(vocab,conditional_probability,prior_probability,conditional_probability_dict,line):
    words = line.split()
    # print(words)
    prob_B = prior_probability['B'].ln()
    prob_E = prior_probability['E'].ln()
    prob_A = prior_probability['A'].ln()
    prob_V = prior_probability['V'].ln()
    for word in words:
        if word in vocab:
            prob_B += (conditional_probability_dict[word]['B'] + 1).ln() - (Decimal(total_B + len(vocab))).ln()
            prob_A += (conditional_probability_dict[word]['A'] + 1).ln() - (Decimal(total_A + len(vocab))).ln()
            prob_V += (conditional_probability_dict[word]['V'] + 1).ln() - (Decimal(total_V + len(vocab))).ln()
            prob_E += (conditional_probability_dict[word]['E'] + 1).ln() - (Decimal(total_E + len(vocab))).ln()
    # print(prob_B,prob_E,prob_A,prob_V)
    if prob_B > prob_E and prob_B > prob_A and prob_B > prob_V:
        return 'B'
    elif prob_E > prob_B and prob_E > prob_A and prob_E > prob_V:
        return 'E'
    elif prob_A > prob_B and prob_A > prob_E and prob_A > prob_V:
        return 'A'
    elif prob_V > prob_B and prob_V > prob_E and prob_V > prob_A:
        return 'V'
    else:
        return 'E'
naive_Bayes(vocab,conditional_probability, prior_probability,conditional_probability_dict, testing)
training_acc = (testing['target'] == testing['prediction']).mean()
print(f"Training acc: {training_acc:.3f}")

1
B A
2
B B
3
B B
4
B B
5
E V
6
E V
7
E E
8
E E
9
B B
10
E E
11
B B
12
E E
13
E E
14
E E
15
E E
16
B B
17
E V
18
E E
19
B B
20
E E
21
B B
22
E E
23
E E
24
B B
25
B B
26
E E
27
B B
28
E E
29
E E
30
B B
31
E E
32
E E
33
B B
34
B B
35
B B
36
E E
37
E E
38
B B
39
B B
40
B B
41
E E
42
E E
43
B B
44
E E
45
B B
46
E E
47
B B
48
E E
49
E E
50
B A
51
E E
52
E E
53
E E
54
B B
55
B B
56
B B
57
B B
58
E E
59
E E
60
B B
61
E E
62
E B
63
B B
64
E V
65
B A
66
B B
67
B B
68
E E
69
E V
70
B B
71
E E
72
B B
73
E E
74
E V
75
E E
76
E E
77
B B
78
B B
79
E E
80
E E
81
B B
82
E E
83
E E
84
E E
85
E V
86
B B
87
E E
88
B B
89
B B
90
E E
91
B B
92
B B
93
E E
94
B B
95
E E
96
B B
97
B B
98
B B
99
B B
100
B B
101
B B
102
B B
103
B B
104
E E
105
B B
106
E E
107
E B
108
E E
109
E E
110
B B
111
B B
112
A A
113
B B
114
E E
115
B B
116
E E
117
E E
118
E E
119
B V
120
E E
121
B B
122
E E
123
B B
124
B B
125
E E
126
E E
127
B B
128
B B
129
E E
130
E E
131
B B
132
E E
133
B B
134
E E
135
E E
136
E B
137
B B
138
B B
139


NameError: name 'validation_set_df' is not defined